try gpt-2

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers torch

In [ ]:
!pip install accelerate -U


In [ ]:
!pip install transformers[torch]


In [5]:
import torch
import numpy as np
import pandas as pd

from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments, GPT2Config
from sklearn.metrics import accuracy_score, roc_auc_score


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [23]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

config = GPT2Config.from_pretrained('gpt2', num_labels=2)
config.pad_token_id = tokenizer.eos_token_id

model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=config)

model = model.to(device)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
testset = pd.read_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/new_essay_val_preprocessed.csv')

test_texts = testset['text'].tolist()
true_labels = testset['label'].values

In [25]:
testset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   text                40 non-null     object 
 1   label               40 non-null     int64  
 2   sentence_variation  40 non-null     float64
 3   %stop_word_total    40 non-null     float64
 4   mean_word_length    40 non-null     float64
 5   %adjectives_total   40 non-null     float64
dtypes: float64(4), int64(1), object(1)
memory usage: 2.0+ KB


In [26]:
from torch.utils.data import DataLoader, TensorDataset

test_encodings  = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(true_labels))
test_loader = DataLoader(test_dataset, batch_size=1)

In [27]:
from sklearn.metrics import (precision_recall_curve, roc_curve, auc, roc_auc_score,
                             precision_score, recall_score, f1_score, accuracy_score,
                             confusion_matrix, ConfusionMatrixDisplay)

# predict
model.eval()
predictions = []
probabilities_list = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(model.device)
        attention_mask = attention_mask.to(model.device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)
        probabilities_list.extend(probs[:, 1].cpu().numpy())  # Get the probability of the positive class
        preds = torch.argmax(logits, dim=1)
        predictions.extend(preds.cpu().numpy())

precision, recall, thresholds = precision_recall_curve(true_labels, probabilities_list)
auprc = auc(recall, precision)
print(f"AUPRC: {auprc:.4f}")
auroc = roc_auc_score(true_labels, probabilities_list)
print(f"AUROC: {auroc:.4f}")
precision_score_val = precision_score(true_labels, predictions)
recall_score_val = recall_score(true_labels, predictions)
f1_score_val = f1_score(true_labels, predictions)
accuracy_score_val = accuracy_score(true_labels, predictions)

print(f"Precision: {precision_score_val:.4f}")
print(f"Recall: {recall_score_val:.4f}")
print(f"F1 Score: {f1_score_val:.4f}")
print(f"Accuracy: {accuracy_score_val:.4f}")

AUPRC: 0.5499
AUROC: 0.5425
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Accuracy: 0.5000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
